<a href="https://colab.research.google.com/github/Tikare234/praveen/blob/master/Shakespeare_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import tensorflow as tf

import numpy as np
import os
import time


In [42]:

path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [43]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [44]:
# Take a look at the first 250 characters in text
print(text[:250])



First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [45]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')


65 unique characters


In [46]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [48]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [49]:
ids = ids_from_chars(chars)
ids


<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [51]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [52]:
chars = chars_from_ids(ids)
chars


<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [53]:
tf.strings.reduce_join(chars, axis=-1).numpy()


array([b'abcdefg', b'xyz'], dtype=object)

In [54]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [55]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [56]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [57]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [58]:
seq_length = 100

In [59]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [60]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [61]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [62]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [63]:
dataset = sequences.map(split_input_target)

In [64]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [65]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [66]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [67]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [68]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [69]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [70]:

model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  16896     
                                                                 
 gru_1 (GRU)                 multiple                  3938304   
                                                                 
 dense_1 (Dense)             multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [71]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [72]:
sampled_indices

array([16, 32, 48, 41, 53, 13, 48, 47, 61, 18, 16, 17, 46,  7,  7, 11,  5,
       32, 31, 37,  4,  4, 57, 65, 57, 61,  9, 32,  3, 40, 64, 28, 20, 28,
       20, 61, 17, 24, 18, 17, 32, 48, 64, 56, 10, 36, 18, 47, 46, 18, 21,
       25, 64, 44, 34,  1, 42, 52, 10, 40,  3, 25, 18, 22, 57, 36, 47, 11,
       57, 30, 11, 19, 32, 29, 34, 46, 60,  7, 12, 57, 12, 44, 51,  7, 13,
       36,  5, 35, 39, 24, 25, 42, 36, 42, 13,  1, 15, 60, 18, 19])

In [73]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b' report.\n\nJULIET:\nThat is no slander, sir, which is a truth;\nAnd what I spake, I spake it to my face'

Next Char Predictions:
 b'CSibn?ihvECDg,,:&SRX$$rzrv.S!ayOGOGvDKEDSiyq3WEhgEHLyeU\ncm3a!LEIrWh:rQ:FSPUgu,;r;el,?W&VZKLcWc?\nBuEF'


In [74]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [75]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1903954, shape=(), dtype=float32)


In [76]:
tf.exp(example_batch_mean_loss).numpy()

66.0489

In [77]:
model.compile(optimizer='adam', loss=loss)

In [78]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [79]:
EPOCHS = 20

In [80]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 14s 56ms/step - loss: 2.7243
Epoch 2/20
172/172 [==============================] - 11s 51ms/step - loss: 1.9961
Epoch 3/20
172/172 [==============================] - 11s 51ms/step - loss: 1.7233
Epoch 4/20
172/172 [==============================] - 11s 52ms/step - loss: 1.5621
Epoch 5/20
172/172 [==============================] - 12s 52ms/step - loss: 1.4626
Epoch 6/20
172/172 [==============================] - 11s 53ms/step - loss: 1.3923
Epoch 7/20
172/172 [==============================] - 11s 52ms/step - loss: 1.3394
Epoch 8/20
172/172 [==============================] - 11s 52ms/step - loss: 1.2956
Epoch 9/20
172/172 [==============================] - 11s 52ms/step - loss: 1.2551
Epoch 10/20
172/172 [==============================] - 11s 53ms/step - loss: 1.2152
Epoch 11/20
172/172 [==============================] - 11s 52ms/step - loss: 1.1786
Epoch 12/20
172/172 [==============================] - 11s 53ms/step - loss: 1.1366
E

In [81]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [82]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [83]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
Harry of Hereford, Lend me and be ascubled
Untold, unbroking-fawns-ellaged so: and wilt thou be made,
Was it not to chide at him.

PARIS:
Go, good Montague, betring 'Shined, you
have not so cledity: if she be our counsel
eyes, when he had changed into a strange eye,
Who was report to hid his full and line of them;
What is thy news, as she's able, they were at Caius Marcius
A season distruction: banished
The metean right royal behalf;
And, if a cur down beloved, and sure,
As beauty use well open'd young moves here?

VALERIA:
It dotes,--

MIRANDA:
Where, by those gallants,
Mistress must be a Richard since thou art.

GLOUCESTER:
I thought or honour, but with him times before,
And made dread spite and hull forget, to their causes
Which makes them makes them against the vester.

QUEEN:
'Tis beauteful tedious, it groans,
Ratcliff, my husband, I hold it there.

PETER:
And so was, I would to God, sir, well, I have quenched him.

TRANTIO:
'Tis more, From fitty.

CAPULET:
Ambleth these ge

In [84]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nThou art a too much less, and for more gracious and these\ngreat very heart, take in her gown, a gentlewoman,\nIf he had sworn to sleep so far assail,\nAnd some revelless cousur'd witch Salors, Henry, both you\nShow me but sometime hence. Then for thee,\nHold Harryine will have God in thee Barner's\nWithollow'd in the one direct barren to thyself.\nThen wast thou in such careful son,\nWhich stands so track, as his fools,\nA hundred years, and a smell-horse is full of\nenjoy! wherefore are gentle livers.\n\nMENENIUS:\nYou have patient tells;\nAnd by a minute of our commandment,\nI mean insolent drop: he fought it;\nHe was to ten two stoof; and every man may know\nWhat sensible of Calibans\nWith oppories impression of Juliet?\nSee, what am I, sir? that, as I intend to do.\n\nKING EDWARD IV:\nTrue arm me, thy good pardon, I thought Claudio\nlevy in your mistress. Thou hast\nYour friends to the service, I'll warm thy parse. What, you news\nUpon your prantician than ha

In [85]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [86]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
Well, let her ersu between as best well
Speak good news! wold Sir Jirn on He,
As he good compasion 
